In [4]:
# 05_tendances_long_terme.ipynb - VERSION CORRIGÉE
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Trends").getOrCreate()

# CORRECTION: Chemins locaux
BASE_PATH = "file:///tmp"
SILVER_NOAA = f"{BASE_PATH}/silver/noaa/"
GOLD_TRENDS = f"{BASE_PATH}/gold/trends/"

# Créer le répertoire de sortie
import os
os.makedirs("/tmp/gold/trends", exist_ok=True)

print("📊 Analyse des tendances long terme...")

# Charger les données Silver NOAA
df = spark.read.parquet(SILVER_NOAA).withColumn("date", F.to_date("date"))

print(f"📈 Données chargées: {df.count()} enregistrements")
print("🔍 Aperçu des données:")
df.select("date", "station_id", "temp_c", "latitude", "longitude").show(5)

# Température moyenne quotidienne par station
print("📅 Calcul des moyennes quotidiennes...")
daily = df.groupBy("date", "station_id").agg(
    F.mean("temp_c").alias("avg_temp"),
    F.count("temp_c").alias("daily_observations")
)

print(f"📊 Jours analysés: {daily.count()}")

# Moyenne mobile sur 30 jours par station
print("📈 Calcul des moyennes mobiles (30 jours)...")
w = Window.partitionBy("station_id").orderBy(F.col("date").cast("long")).rowsBetween(-29, 0)
daily_ma = daily.withColumn("ma30", F.avg("avg_temp").over(w))

# Statistiques annuelles
print("📅 Calcul des statistiques annuelles...")
yearly = df.groupBy(F.year("date").alias("year"), "station_id").agg(
    F.mean("temp_c").alias("year_mean_temp"),
    F.min("temp_c").alias("year_min_temp"),
    F.max("temp_c").alias("year_max_temp"),
    F.count("temp_c").alias("obs_count"),
    F.stddev("temp_c").alias("year_std_temp")
)

# CORRECTION: Tendances décennales - méthode alternative
print("📊 Analyse des tendances décennales...")
decadal = df.withColumn("year_col", F.year("date")) \
           .withColumn("decade", (F.col("year_col") - F.col("year_col") % 10)) \
           .groupBy("decade", "station_id") \
           .agg(
               F.mean("temp_c").alias("decade_mean_temp"),
               F.count("temp_c").alias("decade_obs_count"),
               F.stddev("temp_c").alias("decade_std_temp")
           ).orderBy("decade", "station_id")

# Calcul du réchauffement climatique (pente de régression linéaire)
print("🌡️  Calcul du taux de réchauffement...")

# Préparer les données pour la régression
regression_data = yearly.filter(F.col("obs_count") > 50)  # Au moins 50 observations par an

if regression_data.count() > 0:
    # Calcul de la pente (approximation)
    window_station = Window.partitionBy("station_id").orderBy("year")
    
    warming_rates = regression_data.withColumn("year_index", F.col("year") - F.first("year").over(window_station)) \
                                  .withColumn("temp_diff", F.col("year_mean_temp") - F.first("year_mean_temp").over(window_station)) \
                                  .groupBy("station_id") \
                                  .agg(
                                      F.count("*").alias("years_observed"),
                                      F.avg("year_mean_temp").alias("overall_avg_temp"),
                                      F.max("year_mean_temp").alias("max_year_temp"),
                                      F.min("year_mean_temp").alias("min_year_temp"),
                                      # Approximation de la pente (deg/année)
                                      ((F.max("year_mean_temp") - F.min("year_mean_temp")) / 
                                       (F.max("year") - F.min("year"))).alias("warming_rate_c_per_year")
                                  )
else:
    # Créer un DataFrame vide si pas assez de données
    warming_rates = spark.createDataFrame([], "station_id string, years_observed long, overall_avg_temp double, max_year_temp double, min_year_temp double, warming_rate_c_per_year double")

# Analyses supplémentaires
print("📈 Calcul des tendances saisonnières...")

# Tendances par mois
monthly_trends = df.withColumn("month", F.month("date")) \
                  .groupBy("station_id", "month") \
                  .agg(
                      F.mean("temp_c").alias("monthly_avg_temp"),
                      F.stddev("temp_c").alias("monthly_std_temp"),
                      F.count("*").alias("monthly_obs_count")
                  ).orderBy("station_id", "month")

# Variation interannuelle
print("🔄 Calcul de la variabilité interannuelle...")
yearly_variability = yearly.groupBy("station_id").agg(
    F.stddev("year_mean_temp").alias("interannual_variability"),
    F.count("*").alias("years_recorded"),
    F.avg("year_std_temp").alias("avg_annual_variability")
)

# Sauvegarde des résultats
print("💾 Sauvegarde des tendances...")

# Données quotidiennes avec moyenne mobile
daily_ma.write.mode("overwrite").parquet(GOLD_TRENDS + "/daily_ma")
print(f"✅ Moyennes mobiles sauvegardées: {daily_ma.count()} enregistrements")

# Statistiques annuelles
yearly.write.mode("overwrite").parquet(GOLD_TRENDS + "/yearly")
print(f"✅ Statistiques annuelles sauvegardées: {yearly.count()} enregistrements")

# Tendances décennales
decadal.write.mode("overwrite").parquet(GOLD_TRENDS + "/decadal")
print(f"✅ Tendances décennales sauvegardées: {decadal.count()} enregistrements")

# Taux de réchauffement
warming_rates.write.mode("overwrite").parquet(GOLD_TRENDS + "/warming_rates")
print(f"✅ Taux de réchauffement sauvegardés: {warming_rates.count()} stations")

# Tendances mensuelles
monthly_trends.write.mode("overwrite").parquet(GOLD_TRENDS + "/monthly_trends")
print(f"✅ Tendances mensuelles sauvegardées: {monthly_trends.count()} enregistrements")

# Variabilité interannuelle
yearly_variability.write.mode("overwrite").parquet(GOLD_TRENDS + "/variability")
print(f"✅ Variabilité interannuelle sauvegardée: {yearly_variability.count()} stations")

# Résumé des analyses
print("\n🎉 === ANALYSE DES TENDANCES TERMINÉE ===")

print("\n📊 RÉSUMÉ DES RÉSULTATS:")

print("📅 Couverture temporelle:")
date_range = df.agg(
    F.min("date").alias("start_date"),
    F.max("date").alias("end_date"),
    F.count_distinct("station_id").alias("nb_stations")
).collect()[0]
print(f"   • Période: {date_range['start_date']} à {date_range['end_date']}")
print(f"   • Stations: {date_range['nb_stations']}")

print("\n🌡️  Statistiques annuelles:")
yearly_summary = yearly.agg(
    F.avg("year_mean_temp").alias("avg_annual_temp"),
    F.stddev("year_mean_temp").alias("std_annual_temp"),
    F.min("year_min_temp").alias("global_min_temp"),
    F.max("year_max_temp").alias("global_max_temp")
).collect()[0]
print(f"   • Température annuelle moyenne: {yearly_summary['avg_annual_temp']:.2f}°C")
print(f"   • Variation: {yearly_summary['global_min_temp']:.1f}°C à {yearly_summary['global_max_temp']:.1f}°C")

print("\n📈 Taux de réchauffement:")
if warming_rates.count() > 0:
    warming_stats = warming_rates.agg(
        F.avg("warming_rate_c_per_year").alias("avg_warming_rate"),
        F.min("warming_rate_c_per_year").alias("min_warming_rate"),
        F.max("warming_rate_c_per_year").alias("max_warming_rate")
    ).collect()[0]
    print(f"   • Taux moyen: {warming_stats['avg_warming_rate']:.4f}°C/an")
    print(f"   • Fourchette: {warming_stats['min_warming_rate']:.4f} à {warming_stats['max_warming_rate']:.4f}°C/an")
else:
    print("   • Données insuffisantes pour calculer le réchauffement (période trop courte)")

print("\n📅 Analyse décennale:")
if decadal.count() > 0:
    decade_stats = decadal.agg(
        F.count_distinct("decade").alias("nb_decades"),
        F.min("decade").alias("first_decade"),
        F.max("decade").alias("last_decade")
    ).collect()[0]
    print(f"   • Décades couvertes: {decade_stats['nb_decades']}")
    print(f"   • Période: {decade_stats['first_decade']} - {decade_stats['last_decade']}")
else:
    print("   • Données insuffisantes pour l'analyse décennale")

# Aperçu des données générées
print("\n👀 APERÇU DES DONNÉES GÉNÉRÉES:")

print("Moyennes mobiles (5 premières):")
daily_ma.select("date", "station_id", "avg_temp", "ma30").orderBy("date").show(5)

print("Statistiques annuelles:")
yearly.select("year", "station_id", "year_mean_temp", "year_min_temp", "year_max_temp").orderBy("year").show(5)

print("Tendances mensuelles:")
monthly_trends.select("station_id", "month", "monthly_avg_temp").orderBy("station_id", "month").show(10)

if warming_rates.count() > 0:
    print("Taux de réchauffement par station:")
    warming_rates.select("station_id", "years_observed", "warming_rate_c_per_year").orderBy(F.desc("warming_rate_c_per_year")).show(5)

print(f"\n💾 Toutes les données sauvegardées dans: {GOLD_TRENDS}")

📊 Analyse des tendances long terme...
📈 Données chargées: 100 enregistrements
🔍 Aperçu des données:
+----------+----------+------------------+-----------------+------------------+
|      date|station_id|            temp_c|         latitude|         longitude|
+----------+----------+------------------+-----------------+------------------+
|2024-03-16| STATION_0| 11.23978774675729|40.48186609572492|-74.86755597998558|
|2024-03-17| STATION_1|16.633948794403523|41.71198554111676|-74.15738771337645|
|2024-03-18| STATION_2| 18.97464367553694|41.29907130307006|-74.99322556549149|
|2024-03-19| STATION_3|17.287862514585633|40.97787938028068|-74.75964299884642|
|2024-03-20| STATION_4| 12.29102717093713|39.91442034663697|-74.24808040111922|
+----------+----------+------------------+-----------------+------------------+
only showing top 5 rows

📅 Calcul des moyennes quotidiennes...
📊 Jours analysés: 100
📈 Calcul des moyennes mobiles (30 jours)...
📅 Calcul des statistiques annuelles...
📊 Analyse de